In [1]:
import os
import requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_google_vertexai import ChatVertexAI
load_dotenv()

True

In [2]:
GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# credential json not required if you are working within vertex AI workbench
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/Transcendent/fleet-anagram-244304-7dafcc771b2f.json"

LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # only if you are using text embedding model from google

In [3]:
def get_github_events(username):
    url = f"https://api.github.com/users/{username}/events"
    headers = {"Authorization": f"token {GITHUB_ACCESS_TOKEN}"}

    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors

    events_data = response.json()
    return events_data

In [4]:
# define the chat model
# llm = ChatVertexAI(model="gemini-1.5-flash")
llm = ChatOpenAI(model="gpt-4o")

In [5]:
def summarize_event(event, llm):
    event_details = {
        "type": event["type"],
        "repo": event["repo"]["name"],
        "actor": event["actor"]["login"],
        "created_at": event["created_at"],
        "commit_message": event["payload"]["commits"][0]["message"] if event["type"] == "PushEvent" else None  
    }
    
    # Convert event details to a string format
    event_details_str = "\n".join(f"{key}: {value}" for key, value in event_details.items() if value is not None)
    
    prompt = f"Summarize this GitHub event in one sentence:\n{event_details_str}"
    
    # Generate response from the language model
    response = llm.invoke(prompt)
    
    # Extract the content from the AI's response
    summary = response.content
    return summary


In [6]:
events_data = get_github_events("whanyu1212")
event_summaries = []
for event in events_data:
    if event['type'] in ['PushEvent', 'WatchEvent']:
        summary = summarize_event(event,llm)
        event_summaries.append({"type": event['type'], "summary": summary})

In [13]:
import json
json_string = json.dumps(event_summaries)

In [28]:
response = llm.invoke(f"Tabulate the event type and counts from {event_summaries}. Summarize which repository has the most activity").content


In [24]:
from IPython.display import Markdown, display

In [29]:
display(Markdown(f"<b>{response}</b>"))

<b>Here is the tabulated count of event types:

| Event Type   | Count |
|--------------|-------|
| PushEvent    | 9     |
| WatchEvent   | 14    |

### Summary of Repository Activity

#### PushEvent Details:
- Repository "whanyu1212/Transcendent" had 7 PushEvents:
  - added an example on retrieving github events.
  - minor font changes.
  - add on to the existing todo list.
  - minor font changes.
  - added a simple to do list in readme.
  - README draft.
  - setting up pre-commit hooks.
  - initial dependencies commit.

- Repository "whanyu1212/leetcode-kattis-practice" had 3 PushEvents:
  - adding the minion game and mentioning an unresolved runtime error in some test cases.
  - addressing a directory issue.
  - restructuring the folders.

#### WatchEvent Details:
- Various repositories were watched/starred, with no single repository standing out for most watch events.

### Repository with the Most Activity:
The repository with the most activity is "whanyu1212/Transcendent" with a total of 7 PushEvents.</b>